Assignment 1 - Understanding and Predicting Property Maintenance Fines
This assignment is based on a data challenge from the Michigan Data Science Team (MDST).

The Michigan Data Science Team (MDST) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences (MSSISS) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. Blight violations are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the Detroit Open Data Portal. Only the data already included in your Coursera directory can be used for training the model for this assignment. Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

Building Permits
Trades Permits
Improve Detroit: Submitted Issues
DPD: Citizen Complaints
Parcel Map
We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.


File descriptions (Use only this data for training your model!)

readonly/train.csv - the training set (all tickets issued 2004-2011)
readonly/test.csv - the test set (all tickets issued 2012-2016)
readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
 Note: misspelled addresses may be incorrectly geolocated.

Data fields

train.csv & test.csv

ticket_id - unique identifier for tickets
agency_name - Agency that issued the ticket
inspector_name - Name of inspector that issued the ticket
violator_name - Name of the person/organization that the ticket was issued to
violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
ticket_issued_date - Date and time the ticket was issued
hearing_date - Date and time the violator's hearing was scheduled
violation_code, violation_description - Type of violation
disposition - Judgment and judgement type
fine_amount - Violation fine amount, excluding fees
admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
late_fee - 10% fee assigned to responsible judgments
discount_amount - discount applied, if any
clean_up_cost - DPW clean-up or graffiti removal cost
judgment_amount - Sum of all fines and fees
grafitti_status - Flag for graffiti violations
train.csv only

payment_amount - Amount paid, if any
payment_date - Date payment was made, if it was received
payment_status - Current payment status as of Feb 1 2017
balance_due - Fines and fees still owed
collection_status - Flag for payments in collections
compliance [target variable for prediction] 
 Null = Not responsible
 0 = Responsible, non-compliant
 1 = Responsible, compliant
compliance_detail - More information on why each ticket was marked compliant or non-compliant


In [17]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
train=pd.read_csv('train.csv',engine='python')
test=pd.read_csv('test.csv',engine='python')
train=train.dropna(subset=['compliance'])
le = preprocessing.LabelEncoder()
le.fit(train['violation_code'])
train['violation_code_n']=le.transform(train['violation_code'])
le1 = preprocessing.LabelEncoder()
le1.fit(train['city'])
train['city_n']=le1.transform(train['city'])
#train['city_n']
train.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance,violation_code_n,city_n
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,143,543
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0,74,952
5,22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","KASIMU, UKWELI",6478.0,NORTHFIELD,NaN,2755.0,E. 17TH,LOG BEACH,...,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,143,1930
6,18738,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Deerwood Development Group Inc, Deer",8027.0,BRENTWOOD,NaN,476.0,Garfield,Clinton,...,0.0,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,73,725
7,18735,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,MT ELLIOTT,NaN,8228.0,Mt. Elliott,Detroit,...,0.0,140.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,72,952


In [18]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
train['judgment_amount_n']=scaler.fit_transform(train.judgment_amount.values.reshape(-1, 1))
scaler1 = MinMaxScaler(feature_range=(0,3))
train['city_n1']=scaler1.fit_transform(train.city_n.values.reshape(-1, 1))
scaler2 = MinMaxScaler(feature_range=(0,1))
train['violation_code_n1']=scaler2.fit_transform(train.violation_code_n.values.reshape(-1, 1))
scaler3 = MinMaxScaler(feature_range=(0,1))
train['violation_street_number1']=scaler2.fit_transform(train.violation_street_number.values.reshape(-1, 1))

In [3]:
X=train[['fine_amount','discount_amount','judgment_amount']]


In [4]:
#X=train[['violation_code_n1','violation_street_number1','city_n1','judgment_amount_n']]
X.head()

,fine_amount,discount_amount,judgment_amount
0,250.0,0.0,305.0
1,750.0,0.0,855.0
5,250.0,0.0,305.0
6,750.0,0.0,855.0
7,100.0,0.0,140.0


In [5]:
y=train.compliance



In [6]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)



In [7]:
model=LogisticRegression(C=100).fit(X_train, y_train)

In [8]:
#model=RandomForestClassifier()


In [9]:
#model=GradientBoostingClassifier(learning_rate=1,max_depth=10)
model.fit(X_train,y_train)

LogisticRegression(C=100)

In [10]:
from sklearn.metrics import roc_auc_score,roc_curve,auc
#y_score=model.predict(X_test)
y_score=model.decision_function(X_test)
print(roc_auc_score(y_test,y_score))
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_score)
print(auc(false_positive_rate, true_positive_rate))

0.743506945873291
0.743506945873291


In [11]:
test.set_index('ticket_id',inplace=True)

In [12]:
X_test1=test[['fine_amount','discount_amount','judgment_amount']]
X_test1

,fine_amount,discount_amount,judgment_amount
ticket_id,,,
284932,200.0,0.0,250.0
285362,1000.0,0.0,1130.0
285361,100.0,0.0,140.0
285338,200.0,0.0,250.0
285346,100.0,0.0,140.0
285345,200.0,0.0,250.0
285347,50.0,0.0,85.0
285342,200.0,0.0,230.0
285530,1000.0,0.0,1130.0


In [13]:
compliance=model.predict_proba(X_test1).astype(np.float32)
compliance[:,1].shape

(61001,)

In [14]:
test['compliance']=compliance[:,0]

In [15]:
test.compliance

ticket_id
284932    0.942514
285362    0.996069
285361    0.920909
285338    0.942514
285346    0.920909
285345    0.942514
285347    0.907513
285342    0.800451
285530    0.996069
284989    0.978621
285344    0.907513
285343    0.996069
285340    0.996069
285341    0.907513
285349    0.920909
285348    0.942514
284991    0.978621
285532    0.978621
285406    0.978621
285001    0.978621
285006    0.996069
285405    0.996069
285337    0.978621
285496    0.907513
285497    0.942514
285378    0.996069
285589    0.978621
285585    0.942514
285501    0.920909
285581    0.996069
            ...   
376367    0.990800
376366    0.951113
376362    0.951113
376363    0.942514
376365    0.990800
376364    0.951113
376228    0.951113
376265    0.951113
376286    0.769991
376320    0.951113
376314    0.951113
376327    0.769991
376385    0.769991
376435    0.354288
376370    0.769991
376434    0.907513
376459    0.920909
376478    1.000000
376473    0.951113
376484    0.958483
376482    0.978621
37